# Modèle 1 : Finalisation dernière minute

- On ajoute des poids pour bénéficier "Définition" \*1.3 et réduire "Titre" \*0.7

## Imports

In [ ]:
!pip install pandas spacy bs4 sentence_transformers numpy
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 3.5 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [3]:
import pandas as pd
import re
import unicodedata
import spacy
from bs4 import BeautifulSoup
from spacy.lang.fr.stop_words import STOP_WORDS
import pickle
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.quantization import quantize_embeddings
import numpy as np

/home/patrice/Documents/Etudes/CY-Tech/Ing2_Sem2_2023-2024/UE7_Projet_DataBattle_IA-Pau/ia-pau-data-battle-mars-2024/backend/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Fonctions inchangées

In [5]:
def calculate_average_embedding(text, quantize=False, precision="binary"):
    # Diviser le texte en phrases
    sentences = [sentence.strip() for sentence in text.split('.') if sentence.strip()]
    
    # Calculer l'embedding de chaque phrase
    if quantize :
        sentence_embeddings = model.encode(sentences, precision=precision)
    else :
        sentence_embeddings = model.encode(sentences)
    
    # Prendre la moyenne des embeddings des phrases
    if len(sentence_embeddings) > 0:
        average_embedding = np.mean(sentence_embeddings, axis=0)
    else:
        average_embedding = np.zeros(model.get_sentence_embedding_dimension())
    
    return average_embedding

def pre_processing_sections(sections):
    clean_sections = []

    for texte in sections :
        # Nettoyer HTML Tags
        texte = BeautifulSoup(texte, 'html.parser').get_text()

        # Remplacer "&nbsp;." par rien
        texte = re.sub(r'&nbsp;\.', '', texte)

        # Accents
        texte = unicodedata.normalize('NFD', texte).encode('ascii', 'ignore').decode("utf-8")

        # Retirer les numéros
        texte = re.sub(r'\b\d+\b', '', texte)

        # Tokenization, Lemmatization, Removing Stopwords, Lowercase
        doc = nlp(texte)
        phrases_propres = []
        for phrase in doc.sents:
            tokens = [token.lemma_.lower() for token in phrase if not token.is_stop and not token.is_punct and not token.is_space]
            phrase_propre = ' '.join(tokens)
            if phrase_propre:
                phrases_propres.append(phrase_propre + "")

        # Joining the cleaned sentences back into a single string
        cleaned_text = ' '.join(phrases_propres)

        clean_sections.append(cleaned_text)

    return clean_sections

# NOUVELLE FONCTION
def calculate_section_embedding(sections):
    embeddings = []
    for section in sections :
        section_average_embedding = calculate_average_embedding(section)
        embeddings.append(section_average_embedding)
    return embeddings

def genere_embedding(data, output_file, quantize=False, precision="binary"):

    # Appliquer la fonction pour calculer l'embedding moyen à chaque texte
    if quantize:
        embeddings = data['clean_text'].apply(calculate_section_embedding, quantize=True)
    else :
        embeddings = data['clean_text'].apply(calculate_section_embedding)

    
    # Créer un nouveau DataFrame avec id_solution et les embeddings
    new_data = {
        'id_solution': data['id_solution'],
        'text_embedding': embeddings # Désormais mes embeddings sont des listes de solutions contenant chacune les vecteurs de toutes les phrases de la solution.
    }
    
    # Créer un DataFrame à partir des nouvelles données
    df_embeddings = pd.DataFrame(new_data)
    
    # Storer les embeddings dans un fichier
    with open(output_file, "wb") as fOut:
        pickle.dump(df_embeddings, fOut, protocol=pickle.HIGHEST_PROTOCOL)
    
    print("Les embeddings ont été storer avec succès.")

def find_solution(text_to_compare, embeddings_file, quantize=False, precision="binary"):
    # Calculer l'embedding moyen du texte à comparer
    embedding_to_compare = calculate_average_embedding(text_to_compare, quantize, precision)
    
    # Charger les embeddings à partir du fichier
    with open(embeddings_file, "rb") as fIn:
        df_embeddings = pickle.load(fIn)


    list_similarities = []
    # Pour chaque solution
    for solution in df_embeddings['text_embedding'].values:
        embeddings_array = np.stack(solution)

        # Calculer la similarité cosinus entre l'embedding à comparer et les embeddings dans df_embeddings
        similarities = util.pytorch_cos_sim(embedding_to_compare.reshape(1, -1).astype(np.float64), embeddings_array.astype(np.float64))

        max_value, _ = similarities.max(dim=1)
        list_similarities.append(max_value)

    # Ajouter les similarités au DataFrame df_embeddings
    df_embeddings['similarity'] = list_similarities
    
    # Trier par similarité décroissante
    df_sorted = df_embeddings.sort_values(by='similarity', ascending=False)
    
    # Récupérer les id_solution et les similarités
    solution_info = df_sorted[['id_solution', 'similarity']].head(10)
    
    # Convertir en liste de tuples (id_solution, similarity)
    solution_list = list(zip(solution_info['id_solution'], solution_info['similarity']))
    
    return solution_list

## Fonctions modifiés

### On traite les données

In [49]:
def load_and_merge_data(csv_file='../data/solutions2.csv'):
    dictionnaire_solution = {1: "Titre", 2: "Description", 5: "Application", 6: "Bilan énergétique", 21: "Titre technologie", 22: "Description technologie"}

    # Charger le fichier CSV en spécifiant le séparateur '|'
    df = pd.read_csv(csv_file, sep='|', header=None)

    # Initialiser une liste pour stocker les données de chaque solution
    solutions_data = []

    # Parcourir chaque ligne du DataFrame
    for index, row in df.iterrows():
        id_sol = row[0]
        section = row[1]
        texte = row[2]

        # Vérifier si la section correspond à une clé dans le dictionnaire de solutions
        if section in dictionnaire_solution:
            # Récupérer le nom de la section
            section_name = dictionnaire_solution[section]

            # Chercher si la solution existe déjà dans la liste
            solution_exists = False
            for solution in solutions_data:
                if solution[0] == id_sol:
                    solution_exists = True
                    solution[1][section_name] = texte
                    break

            # Si la solution n'existe pas encore, la créer
            if not solution_exists:
                new_solution = [id_sol, {section_name: texte}]
                solutions_data.append(new_solution)

    return solutions_data

In [94]:
# Appel de la fonction pour obtenir les données
df_solutions = load_and_merge_data()
print(df_solutions[0])

[2, {'Titre technologie': 'Centrale froid', 'Description technologie': 'Centrale froid', 'Titre': 'Installation frigorifique négative de type cascade utilisant du CO2', 'Description': 'Mise en place dune installation frigorifique négative de type cascade utilisant du CO2 comme fluide frigorigène.', 'Application': 'Pour être éligible à CEE, la mise en place doit être effectuée par un professionnel et appliquée dans des locaux de commerce de distribution alimentaire de surface de vente inférieure à 5000 m². Comparé aux autres fluides frigorigènes, le CO2 est un fluide'}]


In [61]:
# Charger le modèle de langue SpaCy
nlp = spacy.load("fr_core_news_sm")

def clean_df_solutions(df_solutions):
    cleaned_data = []
    for item in df_solutions:
        index = item[0]
        solution = item[1]
        cleaned_solution = {}

        for key, value in solution.items():
            if (key == 'Titre' or key == 'Titre technologie') :
                # Pour les titres, ne pas enlever les chiffres
                cleaned_solution[key] = clean_text(str(value), remove_numbers=False)
            else:
                # Pour les autres champs, enlever les chiffres
                cleaned_solution[key] = clean_text(str(value), remove_numbers=True)

        cleaned_data.append([index, cleaned_solution])

    return cleaned_data

def clean_text(text, remove_numbers=True):
    # Nettoyer HTML Tags
    text = BeautifulSoup(text, 'html.parser').get_text()

    # Remplacer "&nbsp;." par rien
    text = re.sub(r'&nbsp;\.', '', text)

    # Supprimer les "l'"
    text = re.sub(r"\bl'", '', text)

    # Accents
    text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode("utf-8")

    if remove_numbers:
        # Retirer les numéros
        text = re.sub(r'\b\d+\b', '', text)

    # Supprimer les caractères seuls
    text = re.sub(r'\b\w\b', '', text)

    # Tokenization, Lemmatization, Removing Stopwords, Lowercase
    doc = nlp(text)
    cleaned_sentences = []
    for sentence in doc.sents:
        tokens = [token.lemma_.lower() for token in sentence if not token.is_stop and not token.is_punct and not token.is_space]
        clean_sentence = ' '.join(tokens)
        if clean_sentence:
            cleaned_sentences.append(clean_sentence)  # Ajouter un point à la fin de la phrase propre

    # Joining the cleaned sentences back into a single string
    cleaned_text = ' '.join(cleaned_sentences)

    return cleaned_text

In [62]:
df_solutions_clean = clean_df_solutions(df_solutions)

/tmp/ipykernel_131978/1373267169.py:25: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()


In [63]:
print(df_solutions_clean)

[[2, {'Titre technologie': 'central froid', 'Description technologie': 'central froid', 'Titre': 'installation frigorifique negativ type cascade utiliser co2', 'Description': 'mise place dun installation frigorifique negativ type cascade utiliser co2 fluide frigorigene', 'Application': 'eligibl cee mise place effectuee professionnel appliquee local commerce distribution alimentaire surface vent inferieure comparer fluide frigorigene co2 fluide'}], [3, {'Titre technologie': 'central froid', 'Description technologie': 'central froid', 'Titre': 'centrale negativ mode booster', 'Description': 'linstallation centrale frigorifique bas moyenne temperatur mode booster consister injecter refoulement compresseur froid negatif laspiration compresseur cycle froid positif maniere rendement cycle frigorifique bas temperature negatif grandement ameliore condensation bas temperature egale temperature devaporation central positif solution diminuer quantite tuyauterie utilisee', 'Application': 'solution

Pour vérifier qu'on garde les numéros

In [101]:
def get_solution_by_id(df_solutions_clean, solution_id):
    for item in df_solutions_clean:
        if item[0] == solution_id:
            return item[1]
    return None

# Utilisation de la fonction pour obtenir le dictionnaire de l'ID 22
solution_id_choisi = 1692
solution_choisie = get_solution_by_id(df_solutions_clean, solution_id_choisi)

# Afficher le dictionnaire de la solution choisie
print(solution_choisie)

{'Titre technologie': 'recyclage dechet suite procede permettre valorisation matier dechet industriel menager', 'Description technologie': 'recyclage dechet suite procede permettre valorisation matier dechet industriel menager', 'Titre': 'reemployer reutiliser materiau inerte chantier chantier'}


### On charge le modèle 

In [67]:
# Charger le modèle
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

## Sauvegarde des embeddings

On va sauvegarder les embeddings en gardant le nom des champs.

In [83]:
def encoder_embeddings(data, model):

    # Fonction pour encoder chaque texte
    def encoder_texte(texte):
        return model.encode(texte)

    # Pour chaque entrée dans les données, encoder tous les champs texte
    for entry in data:
        print("DEBUG : entry = ", entry)
        for champ, valeur in entry[1].items():
            print("DEBUG : champ =", champ, ", valeur = ", valeur)
            # if isinstance(valeur, str):  # S'assurer que la valeur est une chaîne de caractères
            #     # Calculer l'embedding
            #     embedding = encoder_texte(valeur)
            #     # Remplacer le texte par l'embedding
            #     entry[champ] = embedding.tolist()

    return data

Commençons par calculer un embedding sur deux solutions :

In [88]:
solutions = df_solutions_clean[2:3]

data = encoder_embeddings(solutions, model)

DEBUG : entry =  [4, {'Titre technologie': 'ajustement production besoin', 'Description technologie': 'ajustement production besoin', 'Titre': 'arret compresseur week end besoin refroidissement', 'Description': 'lorsquils equiper variation electroniqu vitesse plupart groupe frigorifique fonctionnent rien besoin refroidissement temperature amont compresseur descendre consigne compresseur mettre marche charge nominal besoin refroidissement faible lutilisation compresseur limitee besoin important darreter completement central frigorifique supprimer perte thermique reseau larret manuel groupe frigorifiquer equipe site exister solution automatiser larret dun equipement exemple horloge programmable installee temporisation permettre arret bout dun temps besoin disparaître applicable', 'Application': 'investir echangeur destine refroidir puissance trop petit mettre place horloge programmation hebdomadaire couper compresseur periode dinoccupation', 'Bilan énergétique': 'reseau distribution eten

In [ ]:
embeddings = df_solutions['clean_text'].apply(calculate_section_embedding)
    
# Créer un nouveau DataFrame avec id_solution et les embeddings
new_data = {
    'id_solution': df_solutions['id_solution'],
    'text_embedding': embeddings # Désormais mes embeddings sont des listes de solutions contenant chacune les vecteurs de toutes les phrases de la solution.
}

# Créer un DataFrame à partir des nouvelles données
df_embeddings = pd.DataFrame(new_data)

## Tests

debut debug

In [10]:
df_solutions['clean_text'][0]

['installation frigorifique negativ type cascade utiliser co2',
 'mise place installation frigorifique negativ type cascade utiliser co2 fluide frigorigene',
 'eligibl cee mise place effectuee professionnel appliquee local commerce distribution alimentaire surface vente inferieure m. compare fluide frigorigene co2 fluide']

In [11]:
# Charger le modèle
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

In [16]:
df_solutions = df_solutions[:5]

In [17]:
embeddings = df_solutions['clean_text'].apply(calculate_section_embedding)
    
# Créer un nouveau DataFrame avec id_solution et les embeddings
new_data = {
    'id_solution': df_solutions['id_solution'],
    'text_embedding': embeddings # Désormais mes embeddings sont des listes de solutions contenant chacune les vecteurs de toutes les phrases de la solution.
}

# Créer un DataFrame à partir des nouvelles données
df_embeddings = pd.DataFrame(new_data)

In [28]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

n = 2

print(df_embeddings['id_solution'][n])
for i in range (len(df_embeddings['text_embedding'][n])):
    print(df_embeddings['text_embedding'][n][i])

4
[-1.03763171e-01 -7.16285855e-02 -1.63476672e-02  6.78768605e-02
 -8.02959409e-03  5.81439361e-02 -3.03040873e-02  3.23121212e-02
  4.81155477e-02  1.96518116e-02  2.75819711e-02 -5.73833194e-03
  3.65241505e-02  1.42083149e-02  3.04668560e-04 -6.38149232e-02
 -4.43232618e-02  1.13844663e-01 -6.77734800e-03  4.43911701e-02
  8.71114992e-03  3.50807682e-02  2.83155106e-02  5.14473356e-02
  4.81382245e-03 -1.06881492e-01  5.88109754e-02  4.09756340e-02
  8.38981494e-02 -1.98878162e-02  1.02400035e-01  1.17959604e-01
  6.19233064e-02 -4.35718335e-02  3.09914574e-02 -1.46442289e-02
 -2.59205922e-02  5.52042089e-02 -6.03535101e-02  4.10556979e-02
  3.79193909e-02 -1.28711030e-01  7.90470913e-02  3.45919728e-02
 -1.27998218e-01 -8.40152428e-02 -2.93415952e-02  1.34685065e-03
  3.68741043e-02  9.29871108e-03  2.17929613e-02  1.80443183e-01
 -1.36482388e-01  6.15665093e-02  1.06387846e-01 -1.39018029e-01
 -4.53484580e-02 -5.48882410e-02  1.89979017e-01 -1.09111637e-01
 -5.13241719e-03 -2.542

fin debug